In [91]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import shutil
from collections import OrderedDict

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

from tqdm.notebook import tqdm as tqdm
import pandas as pd
import os
import glob2

In [107]:
res =[]
with open("/FEMTODX/Benchmarking_DL_MIMICIII/Codes/mimic3_mvcv/config/columns_names.txt","r") as f:
    res = f.readlines()

In [104]:
[el.replace("\n","").replace("'",'').replace("""\"""","") for el in res][1]

'gcsverbal_723, 223900'

In [109]:

for f_name in tqdm(glob2.glob("/FEMTODX/Benchmarking_DL_MIMICIII/Codes/mimic3_mvcv/exports-08-05/adm-*.csv")):
    #print(f_name)
    relative_filename = f_name.split("/")[-1]
    df = pd.read_csv(f_name, sep=';')
    df = df.replace(0,np.nan)
    print([col for col in df.columns if '682,224685' in col ])


    #final processing of associated columns
    #old code
    #df["temperature"] = (2*df['body_temperature_1'] + 2*df['body_temperature_2'])/4
    #df["weight"] = (5*df['weight_1'] + 1*df['weight_2'] + 1*df["weight_3"])/7
    #df["height"] = (4*df['height_1'] + 3*df['height_2'])/7
    #df["ie_ratio"] = (df['ie_ratio_1'] + df['ie_ratio_2'])/2
    #print(df.columns)
    #print([col for col in df.columns if 'temperature' in col])

    df["temperature"] = df[['body_temperature_1_678,223761', 'body_temperature_2_676,223762']].mean()
    df["weight"] = df[['weight_1_762, 763, 3723, 3580, 226512', 'weight_2_3581', 'weight_3_3582']].mean()
    df["height"] = df[['height_1_920, 1394, 4187, 3486', 'height_2_3485, 4188, 226707']].mean()
    df["ie_ratio"] = df[['ie_ratio_1_221', 'ie_ratio_2_226873, 226871']].mean()
    
    df['potassium_level_50822, 50971'] = df[['POTASSIUM, WHOLE BLOOD_50822','POTASSIUM_50971']].mean()
    df['sodium_level_50824, 50983'] = df[['SODIUM, WHOLE BLOOD_50824','SODIUM_50983']].mean()
    df['white_blood_cells_count_51300, 51301'] = df[['WBC COUNT_51300','WHITE BLOOD CELLS_51301']].mean()
    df['hgb_50811, 51222'] = df[['HEMOGLOBIN_50811','HEMOGLOBIN_51222']].mean()
    df['chloride_50806, 50902']=  df[['CHLORIDE, WHOLE BLOOD_50806','CHLORIDE_50902']].mean()
    df[[el.replace("\n","").replace("'",'').replace("""\"""","") for el in res]]
    break

['Tidal,Volume,(observed)_682,224685,']



['timestamps_offset_to_the_hour',
 '"gcsverbal_723, 223900"',
 '"gcsmotor_454, 223901"',
 '"gcseyes_184, 220739"',
 '"systolic_blood_pressure_51, 442, 455, 6701, 220050, 220179"',
 '"heart_rate_211, 220045"',
 'pao2_50821',
 '"fio2_50816, 223835,3420, 3422, 190"',
 '"diastolic_blood_pressure_8368, 8440, 8441, 8555, 220180, 220051"',
 '"arterial_pressure_456, 52, 6702, 443, 220052, 220181, 225312"',
 '"respiratory_rate_618, 615, 220210, 224690"',
 '"spo2_peripheral_646, 220277"',
 '"glucose_807,811, 1529, 3745, 3744, 225664, 220621, 226537"',
 'temperature',
 'ie_ratio',
 'weight',
 'height',
 '"urinary_output_40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40086,40096,40651,226559,226560,226561,226584,226563,226564,226565,226567,226557,226558,227488,227489"',
 '"norepinephrine_30047, 30120, 221906"',
 '"epinephrine_30044,30119,30309,221289"',
 '"phenylephrine_30127,30128,221749"',
 '"vasopressin_30051,222315"',
 '"dopamine_30043,30307,221662"',
 '"isuprel_30046,227692

## LOOP DIURETICS

### FUROSEMIDE

In [7]:
sql = '''select distinct  formulary_drug_cd, drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'FURO%';'''

### TORSEMIDE

In [8]:
sql = '''select distinct  formulary_drug_cd, drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'TOR%';'''

### BUMETANIDE

In [9]:
sql = '''select distinct  formulary_drug_cd, drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'BUM%';'''

### ETHACRYNIC ACID

In [10]:
sql = '''select distinct  formulary_drug_cd, drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'ETH%';'''

## POTASSIUM SPARING DIURETICS

### Amiloride

In [11]:
sql = '''select  distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'AMIL%';'''

### Eplerenone (Inspra)

In [12]:
sql = '''select  distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'EPL%';'''

### Spironolactone (Aldactone, Carospir)

In [13]:
sql = '''select distinct  formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'SPIR%';'''

### Triamterene (Dyrenium)

In [14]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'TRIA50%';'''

## ORAL THIAZIDE

### Chlorothiazide (Diuril)

### Neo chlorothiazide: NEOCHL & NEOCHLORO

In [34]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'NEOCHL%'; '''

### Chlorothiazide CHLO250 & CHL500I & CHL500I & CHL500PB

In [15]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where (formulary_drug_cd LIKE 'CHLO250%'  OR formulary_drug_cd LIKE 'CHL500%');'''

### Hydrochlorothiazide (Microzide)

### hydrochlorothiazide : HCTZ25 HCTZ50

In [32]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'HCTZ%';  '''

### Hydrochlorothiazide/Triamterene 37.5/25MG CAP : DIAZ1 (unit = CAP)

In [33]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'DIAZ1%';'''

### Chlorthalidone

In [30]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'CHTH%';'''

### Atenolol-Chlorthalidone

In [31]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where drug LIKE 'Atenolol-Chlorthalidone';'''

### Indapamide

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'LOZ%';'''

# PAINKILLER

### Metolazone

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'metolazon%';'''

### Acetaminophen

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE  'acetaminophen%' ;'''

# NSAIDS

### Naproxen

In [17]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'naproxen%';'''

### Ibuprofen

In [18]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'ibuprofen%';'''

### Flurbiprofen opthalmic FLUR3ES

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'FLUR3ES' ;''' 

### Acetaminophen-Hydrocodone

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'hydrocodone-acetaminophen%';'''

### Acetaminophen-Oxycodone

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'oxycodone-acetaminophen%';'''

### Celecoxib

In [20]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'celecoxib%';'''

### Rofecoxib

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%rofecoxib%';'''

### Parecoxib

In [21]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%arecoxib%';'''

### Valdecoxib

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%valdecoxib%';'''

### Indomethacin

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%indome%';'''

### Ketorolac

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where (formulary_drug_cd LIKE 'KETO15I' OR formulary_drug_cd LIKE 'KETR30I') ;'''

### Ketorolac ophtalmic

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'KETO.5ES' OR LOWER(drug) LIKE 'ketorolac tromethamine';'''

### Sulindac

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'SULI%';'''

### Etodolac

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'ETOD%';'''

### Meloxicam

In [22]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'meloxicam';'''

### Piroxicam

In [23]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'piroxicam';'''

### Nabumetone

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%metone';'''

### Salicylate bismuth

In [28]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where (formulary_drug_cd LIKE 'PEPTO30' OR formulary_drug_cd LIKE 'PEPT240L' OR formulary_drug_cd LIKE 'KAOP228L') ;'''

### Salicylate bismuth chewable

In [29]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where formulary_drug_cd LIKE 'PEPTOBIS' ;'''

### Choline trisalicylate

In [ ]:
sql ='''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where (formulary_drug_cd LIKE 'PEPTO30' OR formulary_drug_cd LIKE 'PEPT240L') ;'''

# PPI

In [ ]:
# aripiprazole

In [42]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%aripiprazole%';'''

In [43]:
# lanzoprazole

In [44]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%lansoprazole%';'''

In [45]:
#omeprazole

In [46]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%omeprazole%';'''

In [47]:
# pantoprazole

In [48]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%pantoprazole%';'''

In [49]:
# dexlansoprazole

In [50]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%dexlansoprazole%';'''

In [51]:
# esomeprazole

In [52]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%esomeprazole%';'''

In [53]:
# rabeprazole

In [54]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%rabeprazole%';'''

# BZD

# Levetiracetam

In [55]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%levetiracetam';'''

# Diazepam

In [57]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%diazepam';'''

# Clonazepam

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%clonazepam';'''

# Clobazam

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%clobazam';'''

# Citalopram

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%citalopram';'''

# Escitalopram

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%escitalopram';'''

# Alprazolam

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%alprazolam';'''

# Estazolam

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%estazolam';'''

# ACE-I

In [ ]:
select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'benazepril'
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'amlodipine-benazepril')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%captopril')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'fosinopril')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where (LOWER(drug) LIKE '%lisinopril' OR LOWER(drug) LIKE '%zestril' ) order by drug)
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'moexipril')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'monopril')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'quinapril')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'ramipril')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'trandolapril')

# Benazepril

In [61]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'benazepril';'''

## Amlodipine-benazepril

In [62]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'amlodipine-benazepril';'''

## Captopril

In [63]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%captopril';'''

## Fosinopril

In [64]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'fosinopril';'''

## Lisinopril

In [65]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where (LOWER(drug) LIKE '%lisinopril' OR LOWER(drug) LIKE '%zestril' ) order by drug;'''

## Moexipril

In [66]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'moexipril';'''

## Monopril

In [67]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'monopril';'''

## Quinapril

In [68]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'quinapril';'''

## Ramipril

In [69]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'ramipril';'''

## Trandolapril

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'trandolapril';'''

# ARBS

## Candesartan

In [78]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%candesartan';'''

## Valsartan

In [79]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%valsartan';'''

## Losartan

In [80]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%losartan';'''

## Irbesartan

In [81]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%irbesartan';'''

## Olmesartan

In [82]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%olmesartan';'''

In [ ]:
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%candesartan')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%valsartan')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%losartan')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%irbesartan')
union
(select subject_id, hadm_id, formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%olmesartan')

In [ ]:
# Anti emetics

In [ ]:
# Kytril

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE 'kytril';'''

In [ ]:
#CHOLERETIC

In [ ]:
# coleveselam

In [ ]:
sql = '''select distinct formulary_drug_cd,drug ,prod_strength,dose_val_rx,dose_unit_rx from prescriptions where LOWER(drug) LIKE '%coleveselam';'''

In [ ]:
#BB